#*Inpainting Code*

In [0]:
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl 

In [0]:
!pip3 install torchvision

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
cd drive/

In [0]:
cd My\ Drive

In [0]:
cd Deep\ Learning\ Project\ 

In [0]:
from skimage.measure import compare_psnr
from __future__ import print_function
import matplotlib.pyplot as plt
%matplotlib inline
import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '1'
import numpy as np
from models.resnet import ResNet
from models.unet import UNet
from models.skip import skip
import torch
import torch.optim
from utils.inpainting_utils import *

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark =True
dtype = torch.cuda.FloatTensor

PLOT = True
imsize = -1
dim_div_by = 64

In [0]:
img_path  = 'data/inpainting/kate.png'     # Add your image path here
mask_path = 'data/inpainting/kate_mask.png' # Add the mask image here 

img_pil, img_np = get_image(img_path, imsize)
img_mask_pil, img_mask_np = get_image(mask_path, imsize)

img_mask_pil = crop_image(img_mask_pil, dim_div_by)
img_pil      = crop_image(img_pil,      dim_div_by)

img_np      = pil_to_np(img_pil)
img_mask_np = pil_to_np(img_mask_pil)

img_pil, img_np = get_image(img_path, imsize)
img_mask_pil, img_mask_np = get_image(mask_path, imsize)

img_mask_var = np_to_torch(img_mask_np).type(dtype)
plot_image_grid([img_np, img_mask_np, img_mask_np*img_np], 3,11);


In [0]:

pad = 'reflection' # 'zero'
OPT_OVER = 'net'
OPTIMIZER = 'adam' 

In [0]:

def repeat_run(mask_path,img_path,i=0,input_depth=32,INPUT='noise',LR=0.1,num_iter=6001, reg_noise_std=0.03,show_every=1000,channels=128,depth=5):


  img_pil, img_np = get_image(img_path, imsize)
  img_mask_pil, img_mask_np = get_image(mask_path, imsize)

  img_mask_pil = crop_image(img_mask_pil, dim_div_by)
  img_pil      = crop_image(img_pil,      dim_div_by)

  img_np      = pil_to_np(img_pil)
  img_mask_np = pil_to_np(img_mask_pil)

  img_pil, img_np = get_image(img_path, imsize)
  img_mask_pil, img_mask_np = get_image(mask_path, imsize)

  img_mask_var = np_to_torch(img_mask_np).type(dtype)
        
  param_noise = False
  figsize = 5
  net = skip(input_depth, img_np.shape[0], 
          num_channels_down = [channels] * depth,
          num_channels_skip = [channels] * depth,
          num_channels_up =   [channels] * depth,  
          filter_size_up = 3, filter_size_down = 3, 
          upsample_mode='nearest', filter_skip_size=1,
          need_sigmoid=True, need_bias=True, pad=pad, act_fun='LeakyReLU').type(dtype)

  net = net.type(dtype)
  net_input = get_noise(input_depth, INPUT, img_np.shape[1:]).type(dtype)

  
  # Compute number of parameters
  s  = sum(np.prod(list(p.size())) for p in net.parameters())
  print ('Number of params: %d' % s)

  # Loss
  mse = torch.nn.MSELoss().type(dtype)

  img_var = np_to_torch(img_np).type(dtype)
  mask_var = np_to_torch(img_mask_np).type(dtype)

  i = 0
  def closure():
      
      global i
      
      if param_noise:
          for n in [x for x in net.parameters() if len(x.size()) == 4]:
              n = n + n.detach().clone().normal_() * n.std() / 50
      
      net_input = net_input_saved
      if reg_noise_std > 0:
          net_input = net_input_saved + (noise.normal_() * reg_noise_std)
          
          
      out = net(net_input)
    
      total_loss = mse(out * mask_var, img_var * mask_var)
      total_loss.backward()
          
      print ('Iteration %05d    Loss %f' % (i, total_loss.item()), '\r', end='')
      if  PLOT and i % show_every == 0:
          out_np = torch_to_np(out)
          plot_image_grid([np.clip(out_np, 0, 1)], factor=figsize, nrow=1)
          
      i += 1
      return total_loss


  net_input_saved = net_input.detach().clone()
  noise = net_input.detach().clone()

  
  p = get_params(OPT_OVER, net, net_input)
  optimize(OPTIMIZER, p, closure, LR, num_iter)

  out_np = torch_to_np(net(net_input))
  return compare_psnr(img_np, out_np) 

We passed different parameters in repeat_run() function to construct the relevant graphs. The existing snippet prints out the PSNR value at different input depth while keeping other parameters of the architecture same.\
To get the output with only certain parameters, we can run repeat_run() only one time with proper image and mask path.\
To examine different padding, optimizers or up-sampling or down-sampling types, we directly changed those paramters in the repeat_run() function.\
We were changing the architecture directly by changing the net in repeat_run() function


In [0]:
from numpy import empty
PSNR_value = empty([10,1])
input_depth_given = [1,2,4,8,16,32,40,64,80,128]
for j_p in range(0,10):
    PSNR_value[j_p]= repeat_run(mask_path,img_path,input_depth=input_depth_given[j_p])